In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_theme()

In [16]:
def make_group(event: str):
  # Посещение корзины с последующим оформлением заказа
  g1 = ['Посещение корзины с добавленными товарами', 'Клик на кнопку Перейти к оформлению', 'Выбор радиобаттона eSIM',
        'Выбор радиобаттона обычной SIM', 'Выбор другого тарифа или номера в корзине']
  
  # Ввод личных данных
  g2 = ['Ввод контактного номера', 'Ввод имени', 'Ввод email', 'Ввод промокода']

  # Оформление доставки
  g3 = ['Клик на кнопку Продолжить 1 (Экран ввода контактных данных)', 'Клик на кнопку Продолжить 2 (Экран выбора региона)',
        'Клик на кнопку Продолжить 3 (Экран выбора способа получения)', 'Клик на кнопку Продолжить 4 (Экран выбора адреса доставки/самовывоза)']

  # Заполнение паспортных данных
  g4 = ['Включение блока паспортных данных', 'Успешное заполнение всех полей паспортных данных']

  if event == 'Покупка':
    return 5
  elif event in g1:
    return 1
  elif event in g2:
    return 2
  elif event in g3:
    return 3
  elif event in g4:
    return 4


In [25]:
def good_events(arr): 
    ''' Функция убирает те действия, которые повторяются. '''
    ans = [arr[0]]  
    for i in range(1, len(arr)):
        if arr[i] != arr[i-1]:
            ans.append(arr[i])
    return ans


def do_voronka(df, delet_koef=0):
    ''' Функция принимает на вход датафрейм и создаёт пользовательские сценарии и считает их кол-во. 
    Есть условный параметри delet_koef - он удаляет те сценарии, которые встерчаются < delet_koef'''
    df = df[['client_id', 'session_id','event_time', 'event_name']]
    df = df.sort_values('event_time').drop('event_time', axis=1)
    df = df.groupby(['client_id', 'session_id'], as_index=False).agg(lambda x: tuple(good_events( x.to_list()) + [0]))
    df['summ'] = 1
    df = df.drop(['client_id', 'session_id'], axis=1)
    df = df.groupby('event_name', as_index=False).agg('sum')[(df['summ'] > delet_koef) | (df.event_name.apply(lambda x: 5 in x))] 
    return df


def do_voronka_uliana(df):
    ''' Функция принимает на вход датафрейм и создаёт словарь, где описывается кол-во последовательностей действий'''
    moves = {i: {k: 0 for k in range(0, 6) if k != i} for i in range(1, 6)}
    df = df[['client_id', 'session_id','event_time', 'event_name']]
    df = df.sort_values('event_time').drop('event_time', axis=1)
    df = df.groupby(['client_id', 'session_id'], as_index=False).agg(lambda x: tuple(good_events(x.to_list()) + [0]))
    df['summ'] = 1
    df = df.drop(['client_id', 'session_id'], axis=1)
    df = df.groupby('event_name', as_index=False).agg('sum')
    for i in range(df.shape[0]):
        event = df.iloc[i]
        events = event.event_name 
        summa = event.summ
        for k in range(len(events)-1):
            first = events[k]
            second = events[k+1]
            moves[first][second] += summa
    return moves

In [26]:
df = pd.read_csv("tele2_dataset.csv", parse_dates=['date', 'event_time'], )

In [27]:
df = df.sort_values('event_time')
df['event_name'] = df.event_name.apply(lambda x: make_group(x))

# Условные обозначения 
-1 - конец сессии

0 - другое

1 - Посещение корзины с последующим оформлением заказа

2 - Ввод личных данных

3 - Оформление доставки

4 - Заполнение паспортных данных

5 - покупка

In [28]:
viborka = df[(df['region_name'] == 'MSK') & (df['rfm_index'] > 222) & (df['campaign'] != '(not set)')]

In [29]:
voronka = do_voronka(df)
voronka.head(5)

,event_name,summ
0,"(1, 0)",20265
1,"(1, 2, 0)",525
2,"(1, 2, 1, 0)",274
3,"(1, 2, 1, 2, 0)",80
4,"(1, 2, 1, 2, 1, 0)",43


In [30]:
voronka = do_voronka_uliana(df)
voronka

{1: {0: 21185, 2: 14683, 3: 2056, 4: 145, 5: 210},
 2: {0: 929, 1: 4246, 3: 14340, 4: 12, 5: 33},
 3: {0: 1433, 1: 1701, 2: 4636, 4: 244, 5: 8487},
 4: {0: 52, 1: 79, 2: 11, 3: 35, 5: 227},
 5: {0: 7047, 1: 1903, 2: 13, 3: 10, 4: 0}}